In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [18]:
def simulate_fleet_evolution(years=30, 
                             start_year=2025, 
                             gas_car_emissions_per_car = lambda year, gas_car_fleet_size: 400, 
                             hybrid_car_emissions_per_car = lambda year, hybrid_car_fleet_size: 200, 
                             ev_car_emissions_per_car = lambda year, ev_car_fleet_size: 50):
    """
    Population-level car fleet model
    
    Parameters:
    - years: number of years to simulate
    - start_year: starting year
    
    Returns:
    - DataFrame with fleet composition over time
    """
        
    # Initial fleet composition (millions of vehicles)
    initial_gas = 12.0      # 80% of 15M total fleet
    initial_hybrid = 2.25   # 15% of 15M total fleet  
    initial_ev = 0.75       # 5% of 15M total fleet
    
    # Annual retirement rates (fraction of fleet retired each year) as functions of year and fleet size (or constants)
    retirement_rate_gas = lambda year, gas_fleet: 1/15 
    retirement_rate_hybrid = lambda year, hybrid_fleet: 1/16
    retirement_rate_ev = lambda year, ev_fleet: 1/18

    # Annual growth rates as function of year and fleet size (or constants)
    growth_rate_gas = lambda year, gas_fleet: -0.03
    growth_rate_hybrid = lambda year, hybrid_fleet: 0.02
    growth_rate_ev = lambda year, ev_fleet: 0.10


    # Initialize parameters
    results = []
    
    gas_fleet = initial_gas
    hybrid_fleet = initial_hybrid  
    ev_fleet = initial_ev
    gas_purchases = 0
    hybrid_purchases = 0
    ev_purchases = 0
    
    for current_year in range(start_year, start_year + years + 1):        
        # Calculate total current fleet
        total_fleet = gas_fleet + hybrid_fleet + ev_fleet

        total_gas_emissions = gas_fleet * gas_car_emissions_per_car(current_year, gas_fleet)
        total_hybrid_emissions = hybrid_fleet * hybrid_car_emissions_per_car(current_year, hybrid_fleet)
        total_ev_emissions = ev_fleet * ev_car_emissions_per_car(current_year, ev_fleet)
        total_emissions = total_gas_emissions + total_hybrid_emissions + total_ev_emissions

        # Store current state
        results.append({
            'year': current_year,
            'gas': gas_fleet,
            'hybrid': hybrid_fleet, 
            'ev': ev_fleet,
            'total': total_fleet,
            'gas_share': gas_fleet / total_fleet,
            'hybrid_share': hybrid_fleet / total_fleet,
            'ev_share': ev_fleet / total_fleet,
            'new_gas_cars': gas_purchases,
            'new_hybrid_cars': hybrid_purchases,
            'new_ev_cars': ev_purchases,
            'gas_emissions': total_gas_emissions,
            'hybrid_emissions': total_hybrid_emissions,
            'ev_emissions': total_ev_emissions,
            'total_emissions': total_emissions

        })
    
        # Retirements
        gas_retired = gas_fleet * retirement_rate_gas(current_year, gas_fleet)
        hybrid_retired = hybrid_fleet * retirement_rate_hybrid(current_year, hybrid_fleet)
        ev_retired = ev_fleet * retirement_rate_ev(current_year, ev_fleet)

        # New purchases
        gas_purchases = gas_fleet * growth_rate_gas(current_year, gas_fleet)
        hybrid_purchases = hybrid_fleet * growth_rate_hybrid(current_year, hybrid_fleet)
        ev_purchases = ev_fleet * growth_rate_ev(current_year, ev_fleet)
        
        # Add new purchases to fleet
        gas_fleet += gas_purchases - gas_retired
        hybrid_fleet += hybrid_purchases - hybrid_retired
        ev_fleet += ev_purchases - ev_retired

    return pd.DataFrame(results)